In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install triton

In [ ]:
!pip install torch==2.3.0+cu121 torchvision==2.3.0+cu121 torchaudio==2.3.0 -f https://download.pytorch.org/whl/cu121/torch_stable.html


Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.2 MB/s eta 0:00:00
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==2.3.0+cu121 (from versions: 0.1.6, 0.2.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2, 0.16.0, 0.16.0+cu121, 0.16.1, 0.16.1+cu121, 0.16.2, 0.16.2+cu121, 0.17.0, 0.17.0+cu121, 0.17.1, 0.17.1+cu121, 0.17.2, 0.17.2+cu121, 0.18.0, 0.18.0+cu121, 0.18.1, 0.18.1+cu121, 0.19.0)
ERROR: No matching distribution found for torchvision==2.3.0+cu121


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
system_message = """
    You are an experienced Named Entity Recognizer.
    Your task is to extract entities from the given text and classify them as either 'core' or 'etc'.


    The reason for extracting 'core' and 'etc' entities is to facilitate query decomposition. By recognizing 'core' and 'etc' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified entities: Super Bowl MVP (core), 1979 (etc), and 1980 (etc).

    - 'core' entities are the main subjects or topics that are consistent across variations of the query.The query will be divided based on this entity.
    - 'etc' entities represent others except for 'core'

    We're going to divide the query based on 'core',
    so it's a condition that one entity has to have a relationship with two or more entities to become 'core'.

    Furthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.
    If it is judged that there is no 'core', you can judge all entities as 'etc'.


    I'll provide the query, the entities, and the relationship between them.
    Write the entities in the following format: EntityName (tag).



    Example:
    Query: Who was the Super Bowl MVP in 1979 and 1980?\nEntity: Super Bowl MVP, 1979, 1980\nSuper Bowl MVP|was|1979\nSuper Bowl MVP|was|1980
    You:
    Super Bowl MVP (core), 1979 (etc), 1980 (etc)

    Example:
    Query: Are Chris Marker and Yvonne Rainer American?\nEntity: Chris Marker, Yvonne Rainer, American\nChris Marker|is|American\nYvonne Rainer|is|American
    You:
    Chris Marker (etc), Yvonne Rainer (etc), American (core)

    Example:
    Query: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?\nEntity: Elon Musk's acquisition, X (formerly Twitter), stock prices, market valuation, Tesla , SpaceX\nElon Musk's acquisition|impacts|stock prices\nElon Musk's acquisition|impacts|market valuation\nElon Musk's acquisition|impacts|Tesla\nElon Musk's acquisition|impacts|SpaceX
    You:
    Elon Musk's acquisition (etc), X (formerly Twitter) (etc), stock prices (etc), market valuation of related companies (core), Tesla (etc), SpaceX (etc)

    Example:
    Query: Who directed The Godfather and Apocalypse Now?\nEntity: Who, The Godfather, Apocalypse Now\nWho|directed|The Godfather\nWho|directed|Apocalypse Now
    You:
    Who (core), The Godfather (etc), Apocalypse Now (etc)

    Treat possessive forms (e.g., Mary's) as part of the same entity.
    Do not add anything else.
    When you present the results, you must present the results in the form of all the entities I gave you. Don't get rid of the entity
    Keep in mind that you can't be a 'core' if you don't get that entity more than once in the relationship I give you

    """

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = system_message
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_data.json")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    #train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/70 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#train model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 70 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.335600
2,2.280200
3,2.611800
4,2.400500
5,2.033000
6,2.191500
7,1.970000
8,1.771800
9,1.783800
10,1.361100


In [ ]:
import re
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "\nYou are an experienced Named Entity Recognizer.\nYour task is to extract entities from the given text and classify them as either 'core' or 'etc'.\n\n\nThe reason for extracting 'core' and 'etc' entities is to facilitate query decomposition. By recognizing 'core' and 'etc' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query \"Who was the Super Bowl MVP in 1979 and 1980?\" can be decomposed into \"Who was the Super Bowl MVP in 1979?\" and \"Who was the Super Bowl MVP in 1980?\" based on the identified entities: Super Bowl MVP (core), 1979 (etc), and 1980 (etc).\n\n- 'core' entities are the main subjects or topics that are consistent across variations of the query.The query will be divided based on this entity.\n- 'etc' entities represent others except for 'core'\n\nWe're going to divide the query based on 'core', \nso it's a condition that one entity has to have a relationship with two or more entities to become 'core'.\n\nFurthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.\nIf it is judged that there is no 'core', you can judge all entities as 'etc'.\n\n\nI'll provide the query, the entities, and the relationship between them.\nWrite the entities in the following format: EntityName (tag).\n\n\n\nExample:\nQuery: Who was the Super Bowl MVP in 1979 and 1980?\nEntity: Super Bowl MVP, 1979, 1980\nSuper Bowl MVP|was|1979\nSuper Bowl MVP|was|1980\nYou:\nSuper Bowl MVP (core), 1979 (etc), 1980 (etc)\n\nExample:\nQuery: Are Chris Marker and Yvonne Rainer American?\nEntity: Chris Marker, Yvonne Rainer, American\nChris Marker|is|American\nYvonne Rainer|is|American\nYou:\nChris Marker (etc), Yvonne Rainer (etc), American (core)\n\nExample:\nQuery: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?\nEntity: Elon Musk's acquisition, X (formerly Twitter), stock prices, market valuation, Tesla , SpaceX\nElon Musk's acquisition|impacts|stock prices\nElon Musk's acquisition|impacts|market valuation\nElon Musk's acquisition|impacts|Tesla\nElon Musk's acquisition|impacts|SpaceX\nYou:\nElon Musk's acquisition (etc), X (formerly Twitter) (etc), stock prices (etc), market valuation of related companies (core), Tesla (etc), SpaceX (etc)\n\nExample:\nQuery: Who directed The Godfather and Apocalypse Now?\nEntity: Who, The Godfather, Apocalypse Now\nWho|directed|The Godfather\nWho|directed|Apocalypse Now\nYou:\nWho (core), The Godfather (etc), Apocalypse Now (etc)\n\nTreat possessive forms (e.g., Mary's) as part of the same entity.\nDo not add anything else.\n\nKeep in mind that you can't be a 'core' if you don't get that entity more than once in the relationship I give you\n", # instruction
        "Were Scott Derrickson and Ed Wood of the same nationality?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
#tokenizer.batch_decode(outputs)

decoded_outputs = tokenizer.batch_decode(outputs)

response_texts = []
for text in decoded_outputs:
    # '### Response:' 이후의 텍스트를 추출합니다.
    response = text.split("### Response:")[1].strip()
    response = re.sub(r'^<|end_of_text|>', '', response)
    response = response.replace('<||', '')
    response_texts.append(response)

# response_texts 리스트를 하나의 문자열로 결합
combined_responses = "\n".join(response_texts)

# 결합된 문자열 출력 (원하는 경우)
print(combined_responses)


Scott Derrickson (etc), Ed Wood (etc), nationality (core)


### **f1 score section**

In [ ]:
import json
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer



In [ ]:
# 테스트 데이터 로드
data = [
    {
        "text": "Were Scott Derrickson and Ed Wood of the same nationality?",
        "gold_ner": "Entities: Scott Derrickson (PER), Ed Wood (PER), same nationality (CONCEPT)\n\nKey Points: Scott Derrickson (difference), Ed Wood (difference), same nationality (core)"
    },
    {
        "text": "What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?",
        "gold_ner": "Entities: government position (CONCEPT), the woman (CONCEPT), Corliss Archer (CONCEPT), the film Kiss and Tell (CONCEPT)\n\nKey Points: government position (core), the woman (core), Corliss Archer (difference), the film Kiss and Tell (71, 91, difference)"
    }
]

In [ ]:
def get_model_response(query):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            system_message, # instruction
            query, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    #tokenizer.batch_decode(outputs)

    decoded_outputs = tokenizer.batch_decode(outputs)

    response_texts = []
    for text in decoded_outputs:
        # '### Response:' 이후의 텍스트를 추출합니다.
        response = text.split("### Response:")[1].strip()
        response = re.sub(r'^<|end_of_text|>', '', response)
        response = response.replace('<||', '')
        response_texts.append(response)

    # response_texts 리스트를 하나의 문자열로 결합
    combined_responses = "\n".join(response_texts)

    return combined_responses

def extract_entities(ner_output):
    # 'Entities:' 뒤의 문자열을 분리해서 엔티티 리스트를 추출
    entities = []
    if ',' in ner_output:
        entities_str = ner_output.split('\n')[-1].strip()  # 가장 마지막 줄을 가져옴
        entities = [tuple(e.strip().rsplit(" ", 1)) for e in entities_str.split(", ")]
    return entities

def get_entities_and_keypoints(query):
    response = get_model_response(query)
    return response

def evaluate_model(test_data):
    y_true = []
    y_pred = []
    incorrect_predictions = []

    for item in test_data:
        query = item["input"]
        gold_ner = item["output"]

        predicted_ner = get_entities_and_keypoints(query)

        gold_entities = extract_entities(gold_ner)
        predicted_entities = extract_entities(predicted_ner)

        y_true.append(set(gold_entities))
        y_pred.append(set(predicted_entities))

        # 예측이 잘못된 경우 기록
        if set(gold_entities) != set(predicted_entities):
            incorrect_predictions.append({
                "query": query,
                "gold_entities": gold_entities,
                "predicted_entities": predicted_entities
            })

    mlb = MultiLabelBinarizer()
    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    f1 = f1_score(y_true_bin, y_pred_bin, average='weighted')

    return f1, incorrect_predictions

# JSON 파일 로드
with open('/content/output_valid.json', 'r') as file:
    data = json.load(file)

# 모델 평가 및 잘못된 예측 가져오기
f1, incorrect_predictions = evaluate_model(data)
print(f"F1 Score: {f1}")

# 잘못된 예측 출력
for incorrect in incorrect_predictions:
    print("Query:", incorrect["query"])
    print("Gold Entities:", incorrect["gold_entities"])
    print("Predicted Entities:", incorrect["predicted_entities"])
    print()

F1 Score: 0.8170476899550728
Query: Query: Which of Tara Strong major voice role in animated series is an American animated television series based on the DC Comics fictional superhero team, the "Teen Titans"?
Entity: animated series, voice role, American animated television series, DC Comics, Teen Titans, Tara Strong, fictional superhero team
Tara Strong|has|voice role
voice role|in|animated series
animated series|is|American animated television series
American animated television series|based on|DC Comics
DC Comics|has|fictional superhero team
fictional superhero team|is|Teen Titans
Gold Entities: [('Tara Strong', '(etc)'), ('voice role', '(etc)'), ('animated series', '(core)'), ('American animated television series', '(etc)'), ('DC Comics', '(etc)'), ('Teen Titans', '(etc)'), ('fictional superhero team', '(etc)')]
Predicted Entities: [('Tara Strong', '(etc)'), ('animated series', '(etc)'), ('voice role', '(etc)'), ('American animated television series', '(etc)'), ('DC Comics', '(etc

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [("nation's history", '(etc)'), ('2012 game', '(etc)'), ('2017 film', '(core)'), ('58th quadrennial American presidential election', '(core)'), ('American multinational confectionery', '(etc)'), ('American multinational equity firm', '(core)'), ('American professional baseball team', '(core)'), ('Andrew Jaspan', '(core)'), ('Argentine PGA Championship record holder', '(core)'), ('Backford Cross', '(core)'), ('British University', '(core)'), ('British singer-songwriter', '(core)'), ('Bubblegum Alley', '(etc)'), ('CEO', '(core)'), ('David Huntsinger', '(core)'), ('English local newspaper', '(etc)'), ('Irish Fine Gael politician', '(etc)'), ('Maxeda', '(etc)'), ('McFarland', '(etc)'), ('Nina', '(etc)'), ('Robert Suettinger', '(core)'), ('Rock singer', '(etc)'), ('SuperMansion', '(core)'), ('Texas Tech', '(etc)'), ('The Leader of Fine Gael', '(core)'), ('The lamp', '(etc)'), ('The yo

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are an experienced Named Entity Recognizer.\nYour task is to extract entities from the given text and classify them as either 'core' or 'difference'.\n- 'core' entities are the main subjects or topics that are consistent across variations of the query.\n- 'difference' entities are the specific attributes or variations that distinguish different instances of the query.\n\nThe reason for extracting 'core' and 'difference' entities is to facilitate query decomposition. By recognizing 'core' and 'difference' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query \"Who was the Super Bowl MVP in 1979 and 1980?\" can be decomposed into \"Who was the Super Bowl MVP in 1979?\" and \"Who was the Super Bowl MVP in 1980?\" based on the identified entities: Super Bowl MVP (core), 1979 (difference), and 1980 (difference).\n\nFurthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.\n\nFor each entity, include the entity name, its position in the sentence, and its tag (either core or difference).\nWrite the entities in the following format: EntityName (start, end, tag).\nMake sure to recognize questions such as 'who' and 'which' and 'what' and 'when' and 'where' as an entity. Except for PER, LOC, and TIME, you can name them as CONCEPT.\n\nIn the <Entities> area, put names for PER, LOC, ORG, TIME, CONCEPT\nIn the <Key points> area, put a name for core and difference.\n\n\n\nExample:\nQuery: Who was the Super Bowl MVP in 1979 and 1980?\nYou:\nEntities: Who(0, 3, CONCEPT), Super Bowl MVP (9, 22, CONCEPT), 1979 (27, 31, TIME), 1980 (36, 40, TIME)\nKey Points:  Who(0, 3, core), Super Bowl MVP (9, 22, core), 1979 (27, 31, difference), 1980 (36, 40, difference)\n\nExample:\nQuery: Are Chris Marker and Yvonne Rainer American?\nYou:\nEntities: Chris Marker (4, 15, PER), Yvonne Rainer (20, 33, PER), American (34, 41, CONCEPT)\nKey Points: Chris Marker (4, 15, difference), Yvonne Rainer (20, 33, difference), American (34, 41, core)\n\nExample:\nQuery: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?\nYou:\nEntities: Elon Musk's acquisition (8, 31, CONCEPT), X (formerly Twitter) (24, 42, ORG), stock prices (54, 64, CONCEPT), market valuation (65, 79, CONCEPT), Tesla (84, 90, ORG), SpaceX (94, 102, ORG)\nKey Points: Elon Musk's acquisition (8, 31, core), X (formerly Twitter) (24, 42, core), stock prices (54, 64, difference), market valuation (65, 79, difference), Tesla (84, 90, difference), SpaceX (94, 102, difference)\n\nExample:\nQuery: Who directed The Godfather and Apocalypse Now?\nYou:\nEntities: Who (0, 3, CONCEPT), The Godfather (13, 25, CONCEPT), Apocalypse Now (30, 44, CONCEPT)\nKey Points: Who (0, 3, core), The Godfather (13, 25, difference), Apocalypse Now (30, 44, difference)\n\nTreat possessive forms (e.g., Mary's) as part of the same entity.\nDo not add anything else.", # instruction
        "Were Scott Derrickson and Ed Wood of the same nationality?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an experienced Named Entity Recognizer.
Your task is to extract entities from the given text and classify them as either 'core' or 'difference'.
- 'core' entities are the main subjects or topics that are consistent across variations of the query.
- 'difference' entities are the specific attributes or variations that distinguish different instances of the query.

The reason for extracting 'core' and 'difference' entities is to facilitate query decomposition. By recognizing 'core' and 'difference' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified 

In [ ]:
# Check if any modules are offloaded to CPU or disk
!nvidia-smi

Tue Aug 27 16:48:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              43W /  70W |   6923MiB / 15360MiB |     64%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--